In [ ]:
import subprocess
import cv2
import pytesseract as tess
from gtts import gTTS
import os

# Function to convert text to speech in the specified language
def text_to_speech(text, lang_code):
    speech = gTTS(text=text, lang=lang_code, tld="co.in")
    speech.save("output.mp3")
    os.system("mpg321 output.mp3")


# ---------------------------------------------------
# Prompt user for language selection
# ---------------------------------------------------
prompt = "Press 1 for English, 2 for Tamil, 3 for Telugu, 4 for Hindi"
text_to_speech(prompt, "en")

temp = int(input("Enter your choice: "))

if temp == 1:
    tess_var = "eng"
    gtts_var = "en"
elif temp == 2:
    tess_var = "tam"
    gtts_var = "ta"
elif temp == 3:
    tess_var = "tel"
    gtts_var = "te"
elif temp == 4:
    tess_var = "hin"
    gtts_var = "hi"
else:
    print("Invalid choice. Defaulting to English.")
    tess_var = "eng"
    gtts_var = "en"


# ---------------------------------------------------
# Capture Image using libcamera-still
# ---------------------------------------------------
output_file = "/home/user/Pictures/test3.jpg"
command = ["libcamera-still", "-o", output_file, "-t", "5000"]  # 5-second timer

try:
    subprocess.run(command, check=True)
    print(f"Image captured and saved to {output_file}")
except subprocess.CalledProcessError as e:
    print(f"Error capturing image: {e}")
    text_to_speech("An error occurred in capturing. Please try again.", gtts_var)
    exit()


# ---------------------------------------------------
# Read & preprocess image
# ---------------------------------------------------
img = cv2.imread(output_file)

if img is None:
    print("Error: Image not found.")
    text_to_speech("Image not found, please try again.", gtts_var)
    exit()

gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
cv2.imwrite(output_file, gray)


# ---------------------------------------------------
# Extract text using Tesseract OCR
# ---------------------------------------------------
try:
    text = tess.image_to_string(gray, lang=tess_var)
    print("Extracted Text:")
    print(text)
except Exception as e:
    print("OCR failed:", e)
    text_to_speech("Image is blurry, try again.", gtts_var)
    exit()


# ---------------------------------------------------
# Convert extracted text to speech
# ---------------------------------------------------
try:
    text_to_speech(text, gtts_var)
except AssertionError:
    print("Image blurry.")
    text_to_speech("Image is blurry, try again.", "en")


# ---------------------------------------------------
# Cleanup
# ---------------------------------------------------
try:
    os.remove("output.mp3")
except:
    pass
